In [92]:
import numpy as np
import galois
GF2 = galois.GF(2)
irreducible_poly = galois.Poly([1,0,0,0,1,1,0,1,1], field = GF2)
Gfield = galois.GF(2**8, irreducible_poly=irreducible_poly)

## Shift rows

In [93]:
def shiftRows(stateMatrix):
    return Gfield(np.array([np.roll(row, shift = -i) for i, row in enumerate(stateMatrix)]))

## Add round key

In [94]:
def addRoundKey(stateMatrix, w, i):
    return stateMatrix + w[i * 4: i*4 + 4, :].transpose()

## Mix columns

In [95]:

def mixColumns(stateMatrix):
    A = Gfield(np.array([[0x02, 0x03, 0x01, 0x01],
                         [0x01, 0x02, 0x03, 0x01],
                         [0x01, 0x01, 0x02, 0x03],
                         [0x03, 0x01, 0x01, 0x02]]))
    return A @ stateMatrix

## Sub bytes

In [96]:
def getIndexFromHex(input):
    return (input & 0xF0) >> 4, input & 0xF

sbox = np.array([
                [0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76],
                [0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0],
                [0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15],
                [0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75],
                [0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84],
                [0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF],
                [0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8],
                [0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2],
                [0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73],
                [0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB],
                [0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79],
                [0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08],
                [0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A],
                [0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E],
                [0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF],
                [0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16],
                ])

def subBytes(stateMatrix):
    for i in range(np.shape(stateMatrix)[0]):
        for j in range(np.shape(stateMatrix)[1]):
            x, y = getIndexFromHex(stateMatrix[i, j])
            stateMatrix[i, j] = sbox[x, y]
    return stateMatrix

ref: https://gist.github.com/bonsaiviking/5571001

## Key expansion

In [97]:
def keyExpansion(key): 
    Nk = 4   #number of columns in state
    Nb = 4   #number of 32 bit words in cipher key
    Nr = 10  #number of rounds
    
    w = np.zeros((44, 4), dtype = int)
    
    w[:Nk, :] = key.reshape(4,4)
    
    i = Nk
    while i < Nb * (Nr + 1):
        temp = Gfield(w[i - 1,:])
        
        if i % Nk == 0:
            afterRotWord = np.roll(temp, -1)
            afterSubWord = subWord(afterRotWord)
            temp = Gfield(afterSubWord) + Gfield(roundConstant(i//Nk))
        w[i, :] = Gfield(w[i - Nk,:]) + temp
        i += 1
    return w

def roundConstant(i):
    if i == 1:
        return Gfield(np.array([0x01, 0x0, 0x0, 0x0]))
    array = Gfield(np.array([0x02, 0x0, 0x0, 0x0]))
    return array**(i-1)

def subWord(vector):
    for i in range(np.shape(vector)[0]):
        x, y = getIndexFromHex(vector[i])
        vector[i] = sbox[x, y]
    return vector

## AES Encrypt

In [98]:
def aesEncrypt(stateMatrix, key):
    w = Gfield(keyExpansion(key))
    stateMatrix = stateMatrix + w[:4,:4].transpose()    
    
    for i in range(1, 10):
        stateMatrix = subBytes(stateMatrix)
        stateMatrix = shiftRows(stateMatrix)
        stateMatrix = mixColumns(stateMatrix)
        stateMatrix = addRoundKey(stateMatrix, w, i)
        
    stateMatrix = subBytes(stateMatrix)
    stateMatrix = shiftRows(stateMatrix)
    stateMatrix += w[10 * 4: 10 * 4 + 4, :].transpose()
    return stateMatrix

## Convert string message to hex

In [99]:
def convertStringMessageToHex(message):
    hexArray = [hex(byte) for byte in message.encode()]
    return np.array(hexArray)

print(convertStringMessageToHex("Alfred"))

['0x41' '0x6c' '0x66' '0x72' '0x65' '0x64']


## Convert bitstream to hex

In [100]:
def convertBitArrayToHexArray(bitArray):
    # Pad to nearest 8 bits
    padding = (8 - len(bitArray) % 8) % 8
    bitArray = bitArray + [0] * padding 
    
    # Convert each 8-bit chunk to a byte
    bytes_out = []
    for i in range(0, len(bitArray), 8):
        byte = bitArray[i : i+8]
        byte_val = int(''.join(str(b) for b in byte), 2)
        bytes_out.append(byte_val)

    # 2. Optionally convert to hex
    byte_array = bytes_out
    hex_array = [hex(b) for b in byte_array]
    return hex_array

convertBitArrayToHexArray([1,0,0,0,0,0,0,0,1])

['0x80', '0x80']

## Convert hexstream to bitstream

In [101]:
def convertHexStreamToBitStream(hexArray):
    bit_list = []
    for byte in hexArray:
        bits = bin(byte)[2:].zfill(8)  # binary string, padded to 8 bits
        bit_list.extend([int(b) for b in bits])
    return bit_list

convertHexStreamToBitStream([0x80, 0x80])

[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

## Counter mode

In [102]:

def counterMode(message, key, nonce):
    temp = message.flatten()
    output = Gfield(np.array([], dtype=int))

    if len(temp) % 16 != 0:
        temp = np.append(temp, np.zeros(16 - len(temp) % 16, dtype=int))

    for i in range(len(temp) // 16):
        nonceAsMatrix = Gfield(np.array(list(nonce.to_bytes(16, 'little'))).reshape(4, 4))
        encrypted = aesEncrypt(nonceAsMatrix, key)

        stateMatrix = temp[i * 16 : (i + 1) * 16].reshape(4, 4)
        output = np.append(output, (Gfield(encrypted) + Gfield(stateMatrix)).flatten())
        nonce += 1

    output = output.flatten()
    output = output[:len(message)]  
    return output





## Test

In [112]:
def printInHex(stateMatrix):
    for row in stateMatrix:
        print(" ".join(f"{val:02x}" for val in row))
    print("\n")
        
def runTest():
    # stateMatrix = Gfield(np.array([[0x32, 0x88, 0x31, 0xe0],
    #                                [0x43, 0x5a, 0x31, 0x37],
    #                                [0xf6, 0x30, 0x98, 0x07],
    #                                [0xa8, 0x8d, 0xa2, 0x34]]))
    
    stateMatrix = Gfield(np.array([[0x41, 0x6c, 0x66, 0x62],
                                   [0x65, 0x64, 0x00, 0x00],
                                   [0x00, 0x00, 0x00, 0x00],
                                   [0x00, 0x00, 0x00, 0x00]]))
    
    testKey = Gfield(np.array([0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]))
    encrypted = aesEncrypt(stateMatrix, testKey)
    printInHex(stateMatrix)
    printInHex(encrypted)

def runCounterModeTest():
    message ="""Brian, top marks for not trying
                So kind of you to bless us with your effortlessness
                We're grateful and so strangely comforted


                And I wonder, are you putting us under?
                'Cause we can't take our eyes off the t-shirt and ties combination
                Well, see you later, innovator

                Some want to kiss, some want to kick you
                There's not a net you couldn't slip through
                Or at least that's the impression I get, 'cause you're smooth and you're wet
                And she's not aware yet, but she's yours
                She'll be saying, "Use me, show me the jacuzzi"
                I imagine that it's there on a plate, your high rendezvous rate
                Means that you'll never be frightened to make 'em wait for a while

                I doubt it's your style not to get
                What you set out to acquire, the eyes are on fire
                You are the unforecasted storm, oh"""
    print("Message:", message)
    testKey = Gfield(np.array([0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]))

    messageAsInt = np.array([byte for byte in message.encode("utf-8")])
    cypherText = counterMode(messageAsInt, testKey, 1)

    cypherTextBytes = bytes([int(x) for x in cypherText])
    cypherTextString = cypherTextBytes.decode('utf-8', errors='replace')
    print("Cypher Text:", cypherTextString)

    decryptedText = counterMode(cypherText, testKey, 1)
    print("Decrypted Text:", decryptedText)
    

    decrypted_bytes = bytes([int(x) for x in decryptedText])
    decrypted_str = decrypted_bytes.decode('utf-8', errors='replace')
    print("Decrypted String:", decrypted_str)
  
    
#runTest()
runCounterModeTest()

Message: Brian, top marks for not trying
                So kind of you to bless us with your effortlessness
                We're grateful and so strangely comforted


                And I wonder, are you putting us under?
                'Cause we can't take our eyes off the t-shirt and ties combination
                Well, see you later, innovator

                Some want to kiss, some want to kick you
                There's not a net you couldn't slip through
                Or at least that's the impression I get, 'cause you're smooth and you're wet
                And she's not aware yet, but she's yours
                She'll be saying, "Use me, show me the jacuzzi"
                I imagine that it's there on a plate, your high rendezvous rate
                Means that you'll never be frightened to make 'em wait for a while

                I doubt it's your style not to get
                What you set out to acquire, the eyes are on fire
                You are the unfo